In [ ]:
import json, os, sys
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
#from utils.logging_config import LOGGER
#from utils.graph_utils import extract_msg_file, append_file, chunk_emails, clean_data, split_email_thread
#from chains.split_emails import SPLIT_EMAILS_CHAIN 

In [ ]:
with open('DATA_TUC-KRITI/123456/123456.json', 'r') as file:
    email_list1 = json.load(file)

with open('src/230009_COMDOCS.json', 'r') as file:
    email_list2 = json.load(file)

with open('src/230009_BN.json', 'r') as file:
    email_list3 = json.load(file)

In [ ]:
def add_to_graph(graph: nx.DiGraph, email_data: List[str], filename: str) -> nx.DiGraph:
    """
    Add the email data to the graph.
    """
    try:
        filename = filename.split('_')[1].split('.')[0]
        previous_key = None
        start = 1

        for n, email in enumerate(email_data):
            for node_key, data in graph.nodes(data=True):
                if data['email_node'] == email:
                    previous_key = node_key
                    start = n

        for n, email in enumerate(email_data[start:], start=start):
            if previous_key is None: 
                ''' Adding new email thread without duplicates '''
                pr_key = n
            else: 
                pr_key = int(previous_key.split("_")[0])
            key = f"{pr_key+1}_{filename}"
            graph.add_node(key, email_node=email)
            if n>1:
                graph.add_edge(previous_key, key)
            previous_key = key
    except Exception as e:
        print(f"Error adding to graph: {e}: from file: {filename}: {email}")
    return graph


In [ ]:
graph = nx.DiGraph()
graph = add_to_graph(graph, email_list1, "123456_PREALERT.msg")
graph = add_to_graph(graph, email_list2, "230009_COMDOCS.json")

In [ ]:
graph = add_to_graph(graph, email_list3, "230009_BN.json")


In [ ]:
plt.figure(figsize=(12, 10))
pos = nx.kamada_kawai_layout(graph)
node_options = {"node_color": "black", "node_size": 10}
edge_options = {"width": 1, "alpha": 0.5, "edge_color": "black", "arrowsize": 5, "connectionstyle": 'arc3,rad=0.2'}
label_options = {"font_size": 5, "font_color": "blue", "verticalalignment": "top", "horizontalalignment": "right"}
nx.draw_networkx_nodes(graph, pos, **node_options)
nx.draw_networkx_edges(graph, pos, **edge_options)
nx.draw_networkx_labels(graph, pos, **label_options)
plt.show()